In [44]:
import xmltodict
import json
import pandas as pd
import numpy as np
import re

with open('consolidated.xml', 'r') as f:
    data = f.read()

cons_dict = xmltodict.parse(data)
cons_json = json.dumps(cons_dict, indent=2)
print(cons_json[:1000])

{
  "CONSOLIDATED_LIST": {
    "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "@xsi:noNamespaceSchemaLocation": "https://www.un.org/sc/resources/sc-sanctions.xsd",
    "@dateGenerated": "2022-10-21T20:40:12.605-04:00",
    "INDIVIDUALS": {
      "INDIVIDUAL": [
        {
          "DATAID": "6908555",
          "VERSIONNUM": "1",
          "FIRST_NAME": "RI",
          "SECOND_NAME": "WON HO",
          "THIRD_NAME": null,
          "UN_LIST_TYPE": "DPRK",
          "REFERENCE_NUMBER": "KPi.033",
          "LISTED_ON": "2016-11-30",
          "COMMENTS1": "Ri Won Ho is a DPRK Ministry of State Security Official stationed in Syria supporting KOMID.",
          "DESIGNATION": {
            "VALUE": "DPRK Ministry of State Security Official"
          },
          "NATIONALITY": {
            "VALUE": "Democratic People's Republic of Korea"
          },
          "LIST_TYPE": {
            "VALUE": "UN List"
          },
          "LAST_DAY_UPDATED": {
            "VALUE":

In [45]:
indivs = cons_dict['CONSOLIDATED_LIST']['INDIVIDUALS']['INDIVIDUAL']
# entities = cons_dict['CONSOLIDATED_LIST']['ENTITIES']['ENTITY']
print(type(indivs))  # list

df_ind_norm = pd.json_normalize(indivs,
                                    meta=[
                                          ['INDIVIDUAL_ADDRESS', 'COUNTRY', 'STATE_PROVINCE','ZIP_CODE', 'NOTE'],
                                          ['INDIVIDUAL_DOCUMENT', 'TYPE_OF_DOCUMENT', 'NUMBER', 'ISSUING_COUNTRY', 'NOTE'],
                                    ['INDIVIDUAL_DATE_OF_BIRTH', 'TYPE_OF_DATE', 'DATE'],
                                    ['INDIVIDUAL_ALIAS', 'QUALITY', 'ALIAS_NAME'],
                                     ['INDIVIDUAL_PLACE_OF_BIRTH','CITY', 'STATE_PROVINCE', 'COUNTRY']
                                          ]                                   
                                   )  #, record_path =['INDIVIDUAL'])


<class 'list'>


In [46]:
entities = cons_dict['CONSOLIDATED_LIST']['ENTITIES']['ENTITY']
ent_json = json.dumps(entities, indent=2)
# print(ent_json[:1000])
df_ent_norm = pd.json_normalize(entities, meta=[
                                            ['ENTITY_ADDRESS','STREET', 'CITY', 'COUNTRY','STATE_PROVINCE','ZIP_CODE', 'NOTE'],
                                            ['ENTITY_ALIAS', 'QUALITY', 'ALIAS_NAME']
])
 
df_ent_norm = df_ent_norm.drop(columns=['SORT_KEY', 'SORT_KEY_LAST_MOD','ENTITY_ADDRESS', 'ENTITY_ALIAS'])
df_ent_norm.info()
df_ent_norm

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DATAID                         253 non-null    object
 1   VERSIONNUM                     253 non-null    object
 2   FIRST_NAME                     253 non-null    object
 3   UN_LIST_TYPE                   253 non-null    object
 4   REFERENCE_NUMBER               253 non-null    object
 5   LISTED_ON                      253 non-null    object
 6   COMMENTS1                      246 non-null    object
 7   LIST_TYPE.VALUE                253 non-null    object
 8   LAST_DAY_UPDATED.VALUE         114 non-null    object
 9   ENTITY_ALIAS.QUALITY           38 non-null     object
 10  ENTITY_ALIAS.ALIAS_NAME        38 non-null     object
 11  ENTITY_ADDRESS.CITY            83 non-null     object
 12  ENTITY_ADDRESS.COUNTRY         118 non-null    object
 13  ENTIT

,DATAID,VERSIONNUM,FIRST_NAME,UN_LIST_TYPE,REFERENCE_NUMBER,LISTED_ON,COMMENTS1,LIST_TYPE.VALUE,LAST_DAY_UPDATED.VALUE,ENTITY_ALIAS.QUALITY,ENTITY_ALIAS.ALIAS_NAME,ENTITY_ADDRESS.CITY,ENTITY_ADDRESS.COUNTRY,ENTITY_ADDRESS.NOTE,ENTITY_ADDRESS.STATE_PROVINCE,NAME_ORIGINAL_SCRIPT,ENTITY_ADDRESS.STREET,ENTITY_ADDRESS.ZIP_CODE,SUBMITTED_ON
0,6908629,1,PROPAGANDA AND AGITATION DEPARTMENT (PAD),DPRK,KPe.053,2017-09-11,The Propaganda and Agitation Department has fu...,UN List,None,None,None,Pyongyang,Democratic People's Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
1,110403,1,7TH OF TIR,Iran,IRe.001,2006-12-23,Subordinate of Defence Industries Organisation...,UN List,2014-12-17,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6908409,1,ABDALLAH AZZAM BRIGADES (AAB),Al-Qaida,QDe.144,2014-09-23,An armed group that has carried out joint atta...,UN List,2019-12-06,NaN,NaN,NaN,NaN,"Operates in Lebanon, Syria and the Arabian Pen...",NaN,NaN,NaN,NaN,NaN
3,113445,1,ABU SAYYAF GROUP,Al-Qaida,QDe.001,2001-10-06,Associated with Jemaah Islamiyah (JI) (QDe.092...,UN List,"[2011-12-13, 2020-11-24]",a.k.a.,Al Harakat Al Islamiyya,NaN,Philippines,NaN,NaN,NaN,NaN,NaN,NaN
4,110402,1,ABZAR BORESH KAVEH CO. (BK CO.),Iran,IRe.002,2008-03-03,Involved in the production of centrifuge compo...,UN List,2014-12-17,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,6908697,1,WEIHAI WORLD-SHIPPING FREIGHT,DPRK,KPe.074,2018-03-30,Ship and commercial manager of the XIN GUANG H...,UN List,"[2018-05-23, 2022-09-14]",None,None,Shandong,China,NaN,NaN,NaN,"419-201, Tongyi Lu, Huancui Qu, Weihai",264200,NaN
249,110328,1,YA MAHDI INDUSTRIES GROUP,Iran,IRe.076,2007-03-24,"Subordinate to AIO, which is involved in inter...",UN List,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,110327,1,YAS AIR,Iran,IRe.077,2012-12-20,"Yas Air is the new name for Pars Air, a compan...",UN List,None,None,None,Tehran,Iran (Islamic Republic of),NaN,NaN,NaN,"Mehrabad International Airport, Next to Termin...",NaN,NaN
251,110326,1,YAZD METALLURGY INDUSTRIES (YMI),Iran,IRe.078,2010-06-09,YMI is a subordinate of DIO. [Old Reference #E...,UN List,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option('max_colwidth', -1)
def replace_target_nulls_with_empty_list(df):
    idx = df['TARGETS'].isna()
    df[idx] = df[idx].apply(lambda x: [])
    df.loc[df['TARGETS'].isnull(),['TARGETS']] = df.loc[df['TARGETS'].isnull(),'TARGETS'].apply(lambda x: [])
    return df

df_ind_norm['TARGETS'] = df_ind_norm.loc[:,'COMMENTS1'].str.findall(r'\(([A-Za-z]{3,4}.\d{3,4})\)', flags=re.IGNORECASE | re.MULTILINE)

df_ent_norm['TARGETS'] = df_ent_norm.loc[:,'COMMENTS1'].str.findall(r'\(([A-Za-z]{3,4}.\d{3,4})\)', flags=re.IGNORECASE | re.MULTILINE)

df_ind_norm = replace_target_nulls_with_empty_list(df_ind_norm)
df_ent_norm = replace_target_nulls_with_empty_list(df_ent_norm)

In [48]:
ti=df_ind_norm['TARGETS']
te=df_ent_norm['TARGETS']

In [49]:
# shorten column names
df_ind_norm.columns = [c.replace("INDIVIDUAL_", "") for c in list(df_ind_norm.columns)]

In [50]:
col_list = list(df_ind_norm.columns)
col_list.remove('TARGETS')

In [51]:
for column in ['THIRD_NAME', 'SECOND_NAME', 'FIRST_NAME']:
    df_ind_norm.loc[:, column].replace(np.nan, '', inplace=True)
    # df_ind_norm.loc[df_ind_norm[column].isnull(),[column]] = df_ind_norm.loc[df_ind_norm[column].isnull(),column].apply(lambda x: '')
    # print(df_ind_norm[column].value_counts())
    
df_ind_norm['CONN_COUNT'] = df_ind_norm['TARGETS'].str.len().astype(int)
df_ent_norm['CONN_COUNT'] = df_ent_norm['TARGETS'].str.len().astype(int)

# print('ind conn count = ', df_ind_norm.CONN_COUNT.sum())  #340
assert(df_ind_norm.CONN_COUNT.sum() == 340)
# print('ent conn count = ', df_ent_norm.CONN_COUNT.sum())  #168
assert(df_ent_norm.CONN_COUNT.sum() == 168)


In [43]:
# check for None in TARGETS; there shouldn't be any at this stage
for i in range(df_ind_norm.shape[0]):
    source = df_ind_norm['REFERENCE_NUMBER'].loc[i]
    if (df_ind_norm['TARGETS'].loc[i] is None):
        raise Exception("none found")
        #df_ind_norm.at[i, 'TARGETS'] = []

# concatenate all the individual names for display
df_ind_norm["NAME"] = df_ind_norm['FIRST_NAME'].astype(str) +" "+ df_ind_norm["SECOND_NAME"] +" "+ df_ind_norm["THIRD_NAME"]



link_list = []

for i in range(df_ind_norm.shape[0]):
    source = df_ind_norm['REFERENCE_NUMBER'].loc[i]
    # if (df_ind_norm['TARGETS'].loc[i] is None):
    #     df_ind_norm.at[i, 'TARGETS'] = []

    if (df_ind_norm['TARGETS'].loc[i] is not None) and (len(df_ind_norm['TARGETS'].loc[i]) > 0):        
        # print(i, target_list.loc[i])
        targets_list = []
        for target in df_ind_norm['TARGETS'].loc[i]:
            # target = target.replace('(','').replace(')','')
            if source == target:
                print('WARNING', source, ' equals ', target)
            targets_list.append(target)
            name = df_ind_norm['NAME'].loc[i] 
            link2 = [df_ind_norm['REFERENCE_NUMBER'].loc[i], target, name]
            link_list.append(link2)
            df_ind_norm.at[i, 'TARGETS'] = targets_list

            # df_ind_norm['TARGETS'].loc[i] = targets_list

print(df_ind_norm['TARGETS'].value_counts())
t = df_ind_norm['TARGETS']





# link_list = []

# for i in range(df_ind_norm.shape[0]):
#     source = df_ind_norm['REFERENCE_NUMBER'].loc[i]
#     # if (df_ind_norm['TARGETS'].loc[i] is None):
#     #     df_ind_norm.at[i, 'TARGETS'] = []

#     if (df_ind_norm['TARGETS'].loc[i] is not None) and (len(df_ind_norm['TARGETS'].loc[i]) > 0):        
#         # print(i, target_list.loc[i])
#         targets_list = []
#         for target in df_ind_norm['TARGETS'].loc[i]:
#             # target = target.replace('(','').replace(')','')
#             if source == target:
#                 print('WARNING', source, ' equals ', target)
#             targets_list.append(target)
#             name = df_ind_norm['NAME'].loc[i] 
#             link2 = [df_ind_norm['REFERENCE_NUMBER'].loc[i], target, name]
#             link_list.append(link2)
#             df_ind_norm.at[i, 'TARGETS'] = targets_list

#             # df_ind_norm['TARGETS'].loc[i] = targets_list

# print(df_ind_norm['TARGETS'].value_counts())
# t = df_ind_norm['TARGETS']


[]                                      477
[QDe.115]                                41
[QDe.137]                                15
[QDe.004]                                11
[TAi.004]                                 9
                                       ... 
[QDe.115, QDe.137, QDi.299, QDi.317]      1
[QDe.134, QDe.141, QDe.115]               1
[QDe.115, QDe.129]                        1
[QDe.129, QDi.006]                        1
[TAi.154]                                 1
Name: TARGETS, Length: 118, dtype: int64


In [ ]:

print(len(edge_list))

df_edges = pd.DataFrame(edge_list, columns=['Source', 'Target', 'SourceName'] )
df_edges.to_csv('un_indiv_edges.csv', index=False)
graph = dict(zip(df_ind_norm.REFERENCE_NUMBER, df_ind_norm.TARGETS))

print(len(graph.keys()))
graph_string = str(graph)
print(graph_string[:400])

# networkx attributes formatting:
# attrs = {0: {'attr1': 20, 'attr2': 'nothing'}, 1: {'attr2': 3}, 2: {'attr1': 42}, 3: {'attr3': 'hello'}, 4: {'attr1': 54, 'attr3': '33'}}
# nx.set_node_attributes(G, attrs)

2


ValueError: Shape of passed values is (2, 1), indices imply (2, 3)

In [ ]:
# make source REFERENCE_NUMBER the dataframe index
def set_index_as_ref_num(df):
    df = df.set_index('REFERENCE_NUMBER')
    print(df_ind_index.index)
    return df


df_ind_norm = set_index_as_ref_num(df_ind_norm)
df_ent_norm = set_index_as_ref_num(df_ind_norm)

# df_ind_index = df_ind_norm.set_index('REFERENCE_NUMBER')
# print(df_ind_index.index)

ind_attr_dict = {}


for i in df_ind_index.index:  # index is 'REFERENCE_NUMBER'
    name_dict = {'NAME': df_ind_index.loc[i, 'NAME']}
    ind_attr_dict[i] = name_dict

#importing pickle module
import pickle
# defining the dictionary
# records = { 
#         "subject1" : 'python', 
#         "subject2" : 'java', 
#         "subject3" : 'ruby', 
#         "subject4" : 'c++',
#         "subject5" : 'c' 
#         }
# Open file in write binary mode
# writing dictionary to file using pickle.dump method
pickle_filename = 'attr_dict.pickle'
with open('attr_dict.pickle', 'wb') as handle:
    pickle.dump(ind_attr_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize) (test)
with open(pickle_filename, 'rb') as handle:
     unserialized_data = pickle.load(handle)

print(ind_attr_dict == unserialized_data)

Index(['KPi.033',       nan,       nan, 'QDi.430', 'KPi.034', 'KPi.035',
       'KPi.036', 'KPi.030', 'KPi.032', 'KPi.031',
       ...
       'CDi.031', 'LYi.020', 'QDi.223', 'IRi.043',       nan, 'LYi.023',
       'LYi.026', 'QDi.426', 'TAi.153', 'QDi.289'],
      dtype='object', name='REFERENCE_NUMBER', length=703)
False


In [ ]:
df_edges = pd.DataFrame(link_list, columns=['Source', 'Target', 'Source_Name'] )
df_edges.to_csv('un_indiv_edges.csv', index=False)
graph = dict(zip(df_ind_norm.REFERENCE_NUMBER, df_ind_norm.TARGETS))
print(len(graph.keys()))

703


In [ ]:
# Add Preprocessors
# feature_eng_ip_pair_cnt = GroupByCountPreprocessor(func_kwargs={"resulting_column_name":"ip_pair_count",
#         "groupby_list":["id_orig_h","id_resp_h"],
#         "count_key":"uid"})
# self.output_obj[kwargs["resulting_column_name"]] = \
#     input_obj.groupby(
#         kwargs["groupby_list"])[kwargs["count_key"]].transform('count')

In [ ]:
# S=pd.Series(['Finland','Colombia','Florida','Japan','Puerto Rico','Russia','france'])
# [itm[0] for itm in S.str.findall('^[Ff].*') if len(itm)>0]

In [ ]:
import networkx as nx
dol = {0: [1]}  # single edge (0,1)
G0 = nx.from_dict_of_lists(dol)
list(G0.edges(data=True))


[(0, 1, {})]

In [ ]:
# G = nx.Graph(graph)  # use Graph constructor
G = nx.from_dict_of_lists(graph)
# list(G.edges)
for n, edge in enumerate(list(G.edges(data=True))):
    if n < 10:
        print(edge)
    else:
        break

('QDi.430', 'QDe.115', {})
('QDi.400', 'QDe.137', {})
('QDi.343', 'QDe.004', {})
('QDi.343', 'QDe.137', {})
('QDi.343', 'QDe.115', {})
('TAi.155', 'TAe.012', {})
('TAi.155', 'TAi.144', {})
('QDi.012', 'QDe.098', {})
('QDi.192', 'QDe.003', {})
('TAi.024', 'TAi.145', {})
